In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
def to_df(directory):
    file_list = [file for file in os.listdir(directory) if file.endswith(".csv")]
    file_list = sorted(file_list, key=lambda x: int(x.split('-')[0]))
    df = []

    for i, filename in enumerate(file_list):
        fl = pd.read_csv(os.path.join(directory, filename))
        fl = fl.drop(fl.columns[0], axis=1)
        fl['extracted_value_original'] = fl['extracted_value'].copy()
        df.append(fl)
    return df


interest_list = to_df('data/BTC/serpapi-data-BTC')

In [ ]:
ratio_list = []
keyword = 'extracted_value'

for i in range(len(interest_list) - 1):
    ratio = float(interest_list[i][keyword].iloc[-1]) / float(interest_list[i + 1][keyword].iloc[0])
    ratio_list.append(ratio)
    print("{} of {}: Ratio = {}, Scale 1st hour of week {} = {}, scale last hour of week {} = {}" \
          .format(i + 1, len(interest_list) - 1, ratio_list[i],
                  i + 1, float(interest_list[i + 1][keyword].iloc[0]),
                  i, float(interest_list[i][keyword].iloc[-1]),
                  )
          )
    interest_list[0]["Scale_{}".format(keyword)] = interest_list[0][keyword]
    interest_list[i + 1]["Scale_{}".format(keyword)] = interest_list[i + 1][keyword].apply(
        lambda x: x * ratio_list[i])
    interest_list[i + 1][keyword] = interest_list[i + 1]["Scale_{}".format(keyword)]

In [ ]:
df = pd.concat(interest_list)

In [ ]:
max_interest = np.max(df["Scale_{}".format(keyword)])
df["Scale_{}".format(keyword)]=df["Scale_{}".format(keyword)]/max_interest*100

In [ ]:
df['timestamp_serpapi'] = pd.to_datetime(df['timestamp'], unit='s')

In [ ]:
df[['timestamp_serpapi', 'extracted_value_original', 'Scale_extracted_value']].to_csv('serpapi-BTC-data.csv', index=False)

In [ ]:
df.head(50)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(df['timestamp_serpapi'],df['Scale_extracted_value'] )